### IMPORT  LIBRARIES

In [ ]:
import cv2
import tkinter as tk
import os
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from keras.models import Sequential
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay

### DATA CLEANING AND PREPROCESSING

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("Leaves",shuffle =True,image_size = (255, 255), batch_size = 32);

### CHECKING CLASSES PRESENT IN DATASET

In [ ]:
dataset.class_names

### COUNT OF IMAGES

In [ ]:
dar="/home/admin1/Downloads/Leaves/"
class_names = sorted(os.listdir(dar))
class_dis = [len(os.listdir(dar + name)) for name in class_names]
fig = px.pie(names=class_names, values=class_dis, hole=0.3)
fig.update_layout({"title":{"text":"No of Images in Each Disease", "x":0.50}})
fig.show()

### PLOTTING THE IMAGES

In [ ]:
plt.figure(figsize=(15,15))
for image_batch, label_batch in dataset.take(1):
  for i in range(30):
    ax = plt.subplot(6,5,i+1)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    plt.title(class_names[label_batch[i]]);
    plt.axis("OFF");

### DEFINING DATA FOR TRAINING, TESTING AND VALIDATION

In [ ]:
def dataset_partition(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size,seed=8)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    
    val_ds = ds.skip(train_size).take(val_size) 
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds,val_ds,test_ds

In [ ]:
train_data, valid_data, test_data = dataset_partition(dataset)

In [ ]:
print("Batches of Training data :- ",len(train_data))

In [ ]:
print("Batches of Validation data :- ",len(valid_data))

In [ ]:
print("Batches of Testing data :- ",len(test_data))

### MODEL BUILDING :- CNN

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(255, 255),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
model = models.Sequential([
    resize_and_rescale,data_augmentation,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=(32, 255, 255, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    #multiclass classification
    layers.Dense(5, activation='softmax'),
])


In [ ]:
model.build(input_shape=(32, 255, 255, 3))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',patience=10,
    mode="max"
)

In [ ]:
history = model.fit(train_data, epochs=10,callbacks=early_stopping, validation_data=valid_data,batch_size=32)

### MODEL EVALUATION

In [ ]:
los,acc = model.evaluate(valid_data);
print("Loss is :-",los)
print("Accuracy is :-",acc)

### ACCURACY OF MODEL

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.title("ACCURACY",color="red")
plt.ylabel("ACCURACY")
plt.xlabel("EPOCH")
plt.legend(['accuracy','val_accuracy'],loc='upper left')
plt.show()

### LOSS OF MODEL

In [ ]:
# loss of the model
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("LOSS",color="red")
plt.ylabel("LOSS")
plt.xlabel("EPOCH")
plt.legend(['loss','val_loss'],loc='upper left')
plt.show()   

In [ ]:
def predict(img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict (img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    return predicted_class

In [ ]:
y_tru=[]
y_predicted=[]
for i in range(len(test_data)):
    for images,labels in test_data.take(1):
        for i in range(32):
            predicted_class= predict(images[i].numpy())
            actual_class = class_names[labels[i]]
            y_tru.append(actual_class)
            y_predicted.append(predicted_class)

In [ ]:
# Prediction of first 10 labels in testing data
y_pre=model.predict(test_data)
y_predict=[np.argmax(i) for i in y_pre]
ls=[]
for i in y_predict[:10]:
    if i==0:
        ls.append("Black Spot")
    elif i==1:
        ls.append("Canker")
    elif i==2:
        ls.append("Greening")
    elif i==3:
        ls.append("Healthy")
    else:
        ls.append("Mealnose")
print("Predicted Classes :-\n",ls)

### CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
c=confusion_matrix(y_tru,y_predicted,labels=dataset.class_names)
display_c_m = ConfusionMatrixDisplay(c, display_labels=dataset.class_names)
# Setting colour map to be used
display_c_m.plot(cmap='OrRd', xticks_rotation=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title('Confusion Matrix', fontsize=24);

In [ ]:
from sklearn.metrics import confusion_matrix
c=confusion_matrix(y_tru,y_predicted,labels=dataset.class_names)
print("Confusion Matrix :-\n",c)
print("\n")

### CLASSIFICATION REPORT

In [ ]:
report = classification_report(y_tru,y_predicted,labels=dataset.class_names,target_names=dataset.class_names)
print("Classification Report:-\n",report);

In [ ]:
for batch_image, batch_label in train_data.take(1):
    first_image = batch_image[0].numpy().astype('uint8')
    first_label = dataset.class_names[batch_label[0]]
    
    print('First image of batch to predict :')
    plt.imshow(first_image)
    print('Actual label : ', first_label)
    
    batch_prediction = model.predict(batch_image)
    prediction = dataset.class_names[np.argmax(batch_prediction[0])]
    print('Predicted label : ', prediction)
    plt.axis('off');
    
    


In [ ]:
model.save('my_model.h5')

## VGG16 MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_shape=(255, 255, 3),
    classes=5
)

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
early_stoppings = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',patience=10,
    mode="max"
)

### MODEL BUILDING :- VGG16

In [ ]:
# fit the model
history_model = model.fit_generator(
  train_data,
  validation_data=valid_data,
  epochs=5,
  steps_per_epoch=len(train_data),
  validation_steps=len(valid_data))

### MODEL EVALUATION

In [ ]:
los,acc = model.evaluate(valid_data);
print("Loss is :-",los)
print("Accuracy is :-",acc)

### ACCURACY OF MODEL

In [ ]:
import matplotlib.pyplot as plt ### visualisation
plt.plot(history_model.history['val_accuracy'])
plt.title("ACCURACY",color="red")
plt.plot(history_model.history['accuracy'])
plt.legend(['val_accuracy',['accuracy']])
plt.show()

### LOSS OF MODEL

In [ ]:
import matplotlib.pyplot as plt ### visualisation
plt.plot(history_model.history['val_loss'])
plt.title("LOSS",color="red")
plt.plot(history_model.history['loss'])
plt.legend(['val_loss','loss'])
plt.show()

In [ ]:
val_loss, val_acc = model.evaluate(test_data)
print("Validation loss:", val_loss)
print("Validation accuracy:", val_acc)

## GUI TKINTER

In [ ]:
form = tk.Tk()
form.geometry("1000x500")
form.title('Citrus Plant Disease Classification and Recommendation System Using Deep Learning')
form.resizable(False, False)
my_font1 = ('times', 18, 'bold',)
my_font2 = ('times', 15, 'bold')
l1 = tk.Label(form, text='Citrus Plant Disease Classification and Recommendation System Using Deep Learning', 
              width=70, font=my_font1, bg='white', fg='green', padx = 10, pady = 5)
l1.grid(row=1, column=1)
b1 = tk.Button(form, text='Upload Leaf Image',
               width=20, command=lambda: upload_file())
b1.grid(row=3, column=1)
l2 = tk.Label(form)
l2.grid(row=5, column=1)
l3 = tk.Label(form)
l3.grid(row=11, column=1)

def upload_file():
    global img
    f_types = [('Image Files', '*.png;*.jpg;*.jpeg')]
    filename = filedialog.askopenfilename(filetypes=f_types)
    img = Image.open(filename)
    img.save("leaf.png")
    img_resized = img.resize((255, 255))
    img = ImageTk.PhotoImage(img_resized)
    l2.config(image=img)
    l2.image = img
    b3 = tk.Button(form, text='Predict',
                   width=20, command=lambda: prediction())
    b3.grid(row=9, column=1)

def prediction():
    model =  models.load_model("my_model.h5")
    class_names = ['Black spot', 'Melanose', 'canker', 'greening', 'healthy']
    img = cv2.imread("leaf.png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (255, 255))
    img = (np.expand_dims(img,0))
    weights = model.predict(img)
    img_prediction = weights.argmax(axis=1)[0]
    prediction = class_names[img_prediction]
    l3.config(text=f"Prediction: {prediction}", font=my_font2, fg="green")    
    l4 = tk.Text(form, wrap=WORD, width=90, height=5, fg="green")
    l4.delete('1.0', END)    
    if prediction != "healthy":
            treatments = read_csv("treatments.csv")       
            treatment = (treatments.loc[treatments['disease'] == prediction]).values.tolist() 
            l3.config(fg="red")    
            l4.insert(INSERT, f"Recommendation(Treatment) : {treatment[0][1]}") 
            l4.config(fg="red")
    else:
          l4.insert(INSERT, f"\t\tNo Disease Detected.")     
    l4.config(state=DISABLED, highlightthickness = 0, borderwidth=0, bg='#f0f0f0')
    l4.grid(row=20, column=1)
           
form.mainloop()